# Use Cases for VIGOR

This notebook is to demonstrate a use case for VIGOR

In [1]:
import csv
from neo4j import GraphDatabase
from neo4j.graph import Node, Relationship
from vigor import Graph, VIGOR, predicates, Visualization, Predicate

## Recommendation Use Case

Uses a sample database provided by Neo4j

In [2]:
vigor = VIGOR()

visualization_preds = {}
for vis, score, stat, min, max in predicates:
    if vis not in visualization_preds:
        visualization_preds[vis] = Visualization(vis)
    visualization_preds[vis].add_predicate(Predicate(stat, min, max, score))

for vis in visualization_preds.values():
    vigor.add_visualization(vis)

In [3]:
queries = {
    "recommendations": [
      {
        "q_type": "Retrieval",
        "question": "What are the names of the top 5 movies with a budget over 100 million dollars?",
        "query": "MATCH (m:Movie) WHERE m.budget > 100000000 RETURN m"
      },
      {
        "q_type": "Aggregation",
        "question": "What are the top 5 movies by revenue released in 1995?",
        "query": "MATCH (m:Movie) WHERE m.year = 1995 AND m.revenue IS NOT NULL RETURN m"
      },
      {
        "q_type": "Pathfinding",
        "question": "What are the names of actors who have acted in more than five movies?",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WITH a, count(m) AS numMovies WHERE numMovies > 5 RETURN a"
      },
       {
        "q_type": "Verbose",
        "question": "Find the names of the top 5 movies with the lowest imdbRating.",
        "query": "MATCH (m:Movie) WHERE m.imdbRating IS NOT NULL RETURN m LIMIT 10"
      }
    ],
}

In [4]:
NEO4J_CONNECTION_URI="neo4j+s://demo.neo4jlabs.com:7687"
csv_filename = "../data/use-cases.csv"

In [7]:
header_written = False

for database in queries:
    driver = GraphDatabase.driver(NEO4J_CONNECTION_URI, auth=(database, database))
    for query in queries[database]:
        G = Graph()

        records, summary, keys = driver.execute_query(query['query'], database_=database)
        for record in records:
            for element in record:
                if isinstance(element, Node):
                    G.add_node(record[0], label=element.labels, attributes=element.items())
                elif isinstance(element, Relationship):
                    G.add_edge(record[0], record[1], type=element.type, attributes=element.items())

        graph_stats = G.get_statistics()

        # Unpack recommendations and scores
        recommendations_with_scores = vigor.recommend_n(graph_stats, 8)
        recommendations = [rec[0].value for rec in recommendations_with_scores]  # Extract recommendations
        scores = [rec[1] for rec in recommendations_with_scores]  # Extract scores

        print("Recommended visualizations:", recommendations)

        # Add recommendations and scores to graph stats
        graph_stats['database_name'] = database
        graph_stats['question'] = query['question']
        graph_stats['query'] = query['query']
        graph_stats['q_type'] = query['q_type']
        graph_stats["rec_1"] = recommendations[0]
        graph_stats["rec_2"] = recommendations[1]
        graph_stats["rec_3"] = recommendations[2]
        graph_stats["score_1"] = scores[0]
        graph_stats["score_2"] = scores[1]
        graph_stats["score_3"] = scores[2]

        # Open the CSV file in append mode
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)

            if not header_written:
                writer.writerow(graph_stats.keys())
                header_written = True

            writer.writerow(graph_stats.values())

Error calculating modularity: Graph must have at least one edge and one node to calculate modularity.
Recommended visualizations: ['Treemap', 'Sunburst', 'BioFabric', 'Quilts', 'Adjacency Matrix', 'Attribute Driven Positioning', 'Semantic Substrates', 'Node-Link']
Error calculating modularity: Graph must have at least one edge and one node to calculate modularity.
Recommended visualizations: ['Treemap', 'Sunburst', 'BioFabric', 'Quilts', 'Adjacency Matrix', 'Attribute Driven Positioning', 'Semantic Substrates', 'Node-Link']
Error calculating modularity: Graph must have at least one edge and one node to calculate modularity.
Recommended visualizations: ['Treemap', 'Sunburst', 'BioFabric', 'Quilts', 'Adjacency Matrix', 'Attribute Driven Positioning', 'Semantic Substrates', 'Node-Link']
Error calculating modularity: Graph must have at least one edge and one node to calculate modularity.
Recommended visualizations: ['BioFabric', 'Quilts', 'Adjacency Matrix', 'Treemap', 'Sunburst', 'Attribu